In [1]:
import os
import glob
import pandas as pd
import tensorflow as tf
import numpy as np

from IPython.display import display, HTML

In [1]:
number_of_crossvalidation_run=10
cwd=os.getcwd()
visual_m_path = os.path.join(cwd,'data/book-cover_mean_feature_vectors.pkl')
textual_m_path = os.path.join(cwd,'data/docs_extracted_features.pkl')
models_folder_name = os.path.join(cwd,'models','artificial_net_visual_modality')
path_to_save_test_results=os.path.join(models_folder_name, 'test_results.pkl')
model_checkpoint_path = os.path.join(cwd,models_folder_name,'gmu.ckpt')
models_folder_name_test = os.path.join(cwd,os.pardir,'text_feature_extraction','models','test',str(number_of_crossvalidation_run))
path_to_preprocessed_texts_test=os.path.join(models_folder_name_test,'book-cover_test_dataset.pkl')
summaries_folder_name=os.path.join(cwd,'summaries','artificial_net_visual_modality')


df_visual_m = pd.read_pickle(visual_m_path)
df_textual_m = pd.read_pickle(textual_m_path)

number_of_recipes=len(df_visual_m)
unique_labels=sorted(set(df_visual_m.mean_vector_labels.values))
number_of_classes=len(unique_labels)
possible_class_indices=list(range(0,number_of_classes))
labels2class_indices=dict(zip(unique_labels,possible_class_indices))
print(labels2class_indices)

Test preparation" : "0" "Calendar":"1" "Comics, graphics & novels" :"2" "Romance" :"3""Computer" "&" "technology ":"4"")
"Cookbooks, food & wine":5"Children books" :6"Science fiction and fantasy":7
"Mystery, thriller & suspense ":8"Travel ":9"Law" :10"Reference" :11
"Parenting and relationship ":12"Engineering & transportation ":13"Medical books ":14
"Crafts ,hobbies & home ":15"Religion and spirituality ":16"Sports and outdoors ":18"Self  help ":19
"Science & math ":20"Biography & memoirs" :21"Business & money ":22
"Health fitness & dieting ":23"History ":24"Teen & young Adults ":2 "Arts & photography":26"Literature & Fiction":27
"Humor & Entertainment ":28"Christian books & Bible ":29 "Political & social sciences":30


In [ ]:
not_normalized_features_visual=df_visual_m.mean_feature_vectors.tolist()
mean_visual=np.mean(not_normalized_features_visual)
print("Mean value: visual features ", mean_visual)
subtracted_mean_visual= (not_normalized_features_visual-mean_visual)
max_visual=np.max(subtracted_mean_visual)
print("Max value in visual feature vectors after subraction of mean value: ", max_visual)
normalized_features_visual = (subtracted_mean_visual/max_visual).tolist()
df_visual_m.mean_feature_vectors=normalized_features_visual

not_normalized_features_textual=df_textual_m.doc_embeddings.tolist()
mean_textual=np.mean(not_normalized_features_textual)
print("Mean value: textual features ", mean_textual)
subtracted_mean_textual= (not_normalized_features_textual-mean_textual)
max_textual=np.max(subtracted_mean_textual)
print("Max value in textual feature vectors after subraction of mean value: ", max_textual)
normalized_features_textual = (subtracted_mean_textual/max_textual).tolist()
df_textual_m.doc_embeddings=normalized_features_textual


df_textual_m


In [4]:
df_preprocessed_texts_test = pd.read_pickle(path_to_preprocessed_texts_test)
test_samples=df_preprocessed_texts_test.index.values.tolist()

all_samples=set(range(0,number_of_recipes))
train_samples=list(all_samples.difference(test_samples))
print("train samples: ", train_samples)
print("test samples: ", test_samples)

train samples:  [1, 2, 3, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 36, 37, 40, 41, 42, 43, 44, 45, 46, 48, 49, 50, 51, 52, 53, 55, 56, 57, 58, 59, 61, 62, 63, 64, 65, 66, 68, 69, 70, 71, 72, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 86, 87, 88, 89, 90, 91, 92, 94, 95, 96, 97, 98, 100, 101, 102, 103, 104, 106, 107, 108, 109, 110, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 124, 126, 127, 128, 129, 130, 131, 132, 134, 135, 136, 137, 139, 140, 141, 142, 143, 144, 145, 146, 148, 149, 150, 151, 153, 154, 155, 156, 158, 159, 160, 161, 162, 164, 165, 167, 168, 169, 170, 171, 173, 174, 175, 176, 178, 179, 180, 181, 183, 184, 185, 186, 187, 188, 189, 191, 192, 193, 194, 195, 196, 198, 199, 200, 201, 203, 204, 205, 206, 207, 208, 209, 211, 212, 213, 214, 215, 216, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 230, 231, 232, 234, 235, 236, 237, 238, 240, 241, 243, 245, 247, 248, 249, 250, 251, 252, 253, 254, 255,

In [5]:
df_visual_m_train=df_visual_m.iloc[train_samples]
df_visual_m_test=df_visual_m.iloc[test_samples]
df_textual_m_train=df_textual_m.iloc[train_samples]
df_textual_m_test=df_textual_m.iloc[test_samples]

visual_m_train_inputs=list(df_visual_m_train.mean_feature_vectors.values)
visual_m_test_inputs=list(df_visual_m_test.mean_feature_vectors.values)

textual_m_train_inputs=list(df_textual_m_train.doc_embeddings.values)
textual_m_test_inputs=list(df_textual_m_test.doc_embeddings.values)

train_correct_class_ids=[labels2class_indices[l] for l in df_visual_m_train.mean_vector_labels]
test_correct_class_ids=[labels2class_indices[l] for l in df_visual_m_test.mean_vector_labels]

number_of_training_samples=len(visual_m_train_inputs)
number_of_test_samples=len(visual_m_test_inputs)
len_of_visual_features_vec=len(visual_m_train_inputs[0])
len_of_textual_features_vec=len(textual_m_train_inputs[0])

print(test_correct_class_ids)
print(np.shape(visual_m_train_inputs))
print(np.shape(textual_m_train_inputs))

[0, 0, 0, 1, 1, 2, 3, 3, 3, 4, 4, 5, 6, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 13, 13, 14, 14, 15, 15, 15, 16, 17, 17, 18, 19, 20, 21, 21, 22, 22, 22, 22, 23, 23, 23, 23, 24, 24, 25, 25, 26, 26]
(264, 202)
(264, 27)


In [6]:
batch_size=4
learning_rate=0.001
hidden_state_dim = 27
number_of_training_iterations=50000
print_valid_every=20
num_repeat_training=10

In [ ]:
df_textual_m_test

In [8]:
def remove_directory_content(path):
    files = glob.glob(path+"/*")
    for f in files:
        os.remove(f)

def create_training_batch():
    inputs_visual=[]
    inputs_textual=[]
    correct_classes=[]
    for i in range(batch_size):
        train_sample_index=np.random.choice(range(0,number_of_training_samples),1)[0]
        inputs_visual.append(visual_m_train_inputs[train_sample_index])
        inputs_textual.append(textual_m_train_inputs[train_sample_index])
        correct_classes.append(train_correct_class_ids[train_sample_index])
    return np.array(inputs_visual),np.array(inputs_textual),np.array(correct_classes)

inputs_visual,inputs_textual,correct_classes=create_training_batch()
print(np.array(correct_classes))
print(np.shape(inputs_visual))
print(np.shape(inputs_textual))
print(np.shape(correct_classes))

[24  0  5 20]
(4, 202)
(4, 27)
(4,)


In [9]:
visual = tf.placeholder(tf.float32, shape=[None,len_of_visual_features_vec],name="input_visual_modality")
target = tf.placeholder(tf.int32, shape=[None],name="input_correct_labels")

h_v = tf.layers.dense(visual, hidden_state_dim, activation=tf.nn.tanh, name="h_v")

logits = tf.layers.dense(h_v, number_of_classes, name="h")
scores = tf.nn.sigmoid(logits)

multi_class_labels=tf.one_hot(target, depth=number_of_classes)
loss = tf.reduce_mean(tf.losses.sigmoid_cross_entropy(multi_class_labels=multi_class_labels,
                                       logits=logits))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss)

correct_prediction=tf.equal(tf.argmax(logits, axis=1), tf.argmax(multi_class_labels,axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.name_scope("performance"):
    loss_ph = tf.placeholder(tf.float32,shape=None,name='loss_summary')
    loss_summary = tf.summary.scalar('loss', loss_ph)
    accuracy_ph = tf.placeholder(tf.float32,shape=None,name='accuracy_summary')
    accuracy_summary = tf.summary.scalar('accuracy', accuracy_ph)

saver = tf.train.Saver()

In [10]:
best_accuracy=0

def show_validation_result(session,accuracy_res, loss_res, scores_res):
    print("Validation: Loss: ", loss_res," Accuracy: ", accuracy_res)
    global best_accuracy
    if accuracy_res > best_accuracy:
        best_accuracy=accuracy_res
        df_performance=pd.DataFrame(data={'Accuracy':[accuracy_res], 'Loss':[loss_res]})
        class_scores=list(np.around(scores_res,4))
        predicted_labels = [np.argmax(one_recipe_class_scores) for one_recipe_class_scores in class_scores]
        df_scores=pd.DataFrame(data={'Class scores':class_scores,
                               'Predicted labels':predicted_labels,
                               'Correct labels':test_correct_class_ids,
                               })
        df_res=pd.concat([df_scores, df_performance], axis=1)
        df_res.to_pickle(path_to_save_test_results)
        saver.save(session, model_checkpoint_path)
        display(df_res)
        
    

def train():
        session=tf.InteractiveSession()
        summ_writer = tf.summary.FileWriter(summaries_folder_name, session.graph)
        session.run(tf.global_variables_initializer())
    
        print("Start model training")
    
        for train_iter in range(number_of_training_iterations):           
            inputs_visual,inputs_textual,correct_classes=create_training_batch()
            _, l = session.run([train_op, loss], {visual: inputs_visual,
                                           target: correct_classes})
            print(train_iter, ": Training: loss: ", l)
            summ_loss = session.run(loss_summary, feed_dict={loss_ph:l})
            summ_writer.add_summary(summ_loss, train_iter)
        
            if (train_iter)%print_valid_every==0:                        
                accuracy_res, loss_res, scores_res = session.run([accuracy, loss, scores],
                                                                          {visual: visual_m_test_inputs,
                                                                           target: test_correct_class_ids})        
                
                show_validation_result(session,accuracy_res, loss_res, scores_res)
                summ_accuracy = session.run(accuracy_summary, feed_dict={accuracy_ph:accuracy_res})
                summ_writer.add_summary(summ_accuracy, train_iter)
                
                if accuracy_res==1.0:
                    return 0
                
        session.close()
        return -1

In [11]:
remove_directory_content(summaries_folder_name)
train()

Start model training
0 : Training: loss:  0.6908216
Validation: Loss:  0.6893656  Accuracy:  0.03846154


,Class scores,Correct labels,Predicted labels,Accuracy,Loss
0,"[0.5019, 0.5058, 0.4965, 0.5058, 0.496, 0.5028...",0,9,0.038462,0.689366
1,"[0.4977, 0.4866, 0.4928, 0.5094, 0.5, 0.5112, ...",0,5,NaN,NaN
2,"[0.4943, 0.496, 0.4974, 0.5058, 0.491, 0.4979,...",0,11,NaN,NaN
3,"[0.486, 0.4971, 0.4981, 0.5208, 0.4837, 0.5069...",1,11,NaN,NaN
4,"[0.4975, 0.5027, 0.4989, 0.5023, 0.494, 0.4983...",1,18,NaN,NaN
5,"[0.5004, 0.5145, 0.4991, 0.5061, 0.4928, 0.496...",2,23,NaN,NaN
6,"[0.5136, 0.5072, 0.4842, 0.502, 0.4907, 0.4928...",3,0,NaN,NaN
7,"[0.5098, 0.5118, 0.489, 0.4871, 0.4909, 0.4854...",3,1,NaN,NaN
8,"[0.515, 0.4946, 0.4952, 0.5021, 0.5059, 0.5035...",3,0,NaN,NaN
9,"[0.4803, 0.4934, 0.5001, 0.5137, 0.4989, 0.508...",4,26,NaN,NaN


1 : Training: loss:  0.6898213
2 : Training: loss:  0.68347794
3 : Training: loss:  0.6836042
4 : Training: loss:  0.68499714
5 : Training: loss:  0.6798108
6 : Training: loss:  0.6784371
7 : Training: loss:  0.68074363
8 : Training: loss:  0.6757576
9 : Training: loss:  0.671802
10 : Training: loss:  0.67115885
11 : Training: loss:  0.67638326
12 : Training: loss:  0.6653495
13 : Training: loss:  0.66949534
14 : Training: loss:  0.6709995
15 : Training: loss:  0.6651725
16 : Training: loss:  0.65573657
17 : Training: loss:  0.6561366
18 : Training: loss:  0.64592195
19 : Training: loss:  0.6469886
20 : Training: loss:  0.64787096
Validation: Loss:  0.6476101  Accuracy:  0.01923077
21 : Training: loss:  0.6604279
22 : Training: loss:  0.64713234
23 : Training: loss:  0.6511235
24 : Training: loss:  0.64668494
25 : Training: loss:  0.6263825
26 : Training: loss:  0.6353888
27 : Training: loss:  0.62696904
28 : Training: loss:  0.60639334
29 : Training: loss:  0.6292714
30 : Training: lo

,Class scores,Correct labels,Predicted labels,Accuracy,Loss
0,"[0.0963, 0.099, 0.0839, 0.1144, 0.1422, 0.1102...",0,6,0.057692,0.17817
1,"[0.0713, 0.0713, 0.0607, 0.089, 0.115, 0.0876,...",0,6,NaN,NaN
2,"[0.0856, 0.0888, 0.0758, 0.1048, 0.1276, 0.099...",0,6,NaN,NaN
3,"[0.0461, 0.0495, 0.0401, 0.0634, 0.0798, 0.059...",1,6,NaN,NaN
4,"[0.0727, 0.0761, 0.0632, 0.0892, 0.1139, 0.085...",1,6,NaN,NaN
5,"[0.1124, 0.1215, 0.101, 0.1365, 0.1586, 0.1274...",2,6,NaN,NaN
6,"[0.0866, 0.0891, 0.0688, 0.1069, 0.1249, 0.096...",3,6,NaN,NaN
7,"[0.101, 0.1067, 0.0832, 0.1182, 0.1408, 0.1104...",3,6,NaN,NaN
8,"[0.1606, 0.1526, 0.1385, 0.1732, 0.2058, 0.170...",3,6,NaN,NaN
9,"[0.0449, 0.0482, 0.0397, 0.0616, 0.0848, 0.060...",4,6,NaN,NaN


201 : Training: loss:  0.17021154
202 : Training: loss:  0.17164552
203 : Training: loss:  0.18251391
204 : Training: loss:  0.16454652
205 : Training: loss:  0.17363514
206 : Training: loss:  0.16878739
207 : Training: loss:  0.17801389
208 : Training: loss:  0.17658672
209 : Training: loss:  0.18726829
210 : Training: loss:  0.1651471
211 : Training: loss:  0.17784224
212 : Training: loss:  0.19018745
213 : Training: loss:  0.16956072
214 : Training: loss:  0.18292426
215 : Training: loss:  0.18153802
216 : Training: loss:  0.16557309
217 : Training: loss:  0.16421461
218 : Training: loss:  0.16447726
219 : Training: loss:  0.17014113
220 : Training: loss:  0.18588293
Validation: Loss:  0.17158408  Accuracy:  0.057692308
221 : Training: loss:  0.19819134
222 : Training: loss:  0.17334273
223 : Training: loss:  0.16494499
224 : Training: loss:  0.17096475
225 : Training: loss:  0.17359728
226 : Training: loss:  0.16665821
227 : Training: loss:  0.18440226
228 : Training: loss:  0.1634

...

10917 : Training: loss:  0.021210983
10918 : Training: loss:  0.0064920927
10919 : Training: loss:  0.033183675
10920 : Training: loss:  0.024568748
Validation: Loss:  0.053310372  Accuracy:  0.8076923
10921 : Training: loss:  0.035096914
10922 : Training: loss:  0.009437134
10923 : Training: loss:  0.014639435
10924 : Training: loss:  0.03815188
10925 : Training: loss:  0.026090264
10926 : Training: loss:  0.0064178887
10927 : Training: loss:  0.014325089
10928 : Training: loss:  0.03375739
10929 : Training: loss:  0.009142403
10930 : Training: loss:  0.021133665
10931 : Training: loss:  0.028453322
10932 : Training: loss:  0.019997599
10933 : Training: loss:  0.032654688
10934 : Training: loss:  0.017631328
10935 : Training: loss:  0.016801702
10936 : Training: loss:  0.010964195
10937 : Training: loss:  0.020027328
10938 : Training: loss:  0.018955251
10939 : Training: loss:  0.0059610927
10940 : Training: loss:  0.032674957
Validation: Loss:  0.053516388  Accuracy:  0.8076923
10941

11134 : Training: loss:  0.028498044
11135 : Training: loss:  0.01878512
11136 : Training: loss:  0.03995829
11137 : Training: loss:  0.020621577
11138 : Training: loss:  0.0220388
11139 : Training: loss:  0.029159121
11140 : Training: loss:  0.021806553
Validation: Loss:  0.052510317  Accuracy:  0.7692308
11141 : Training: loss:  0.037311204
11142 : Training: loss:  0.021718418
11143 : Training: loss:  0.04050179
11144 : Training: loss:  0.005080045
11145 : Training: loss:  0.049857777
11146 : Training: loss:  0.026892463
11147 : Training: loss:  0.008504586
11148 : Training: loss:  0.03738442
11149 : Training: loss:  0.020124404
11150 : Training: loss:  0.020695023
11151 : Training: loss:  0.04439973
11152 : Training: loss:  0.01627616
11153 : Training: loss:  0.032787148
11154 : Training: loss:  0.01773286
11155 : Training: loss:  0.051280465
11156 : Training: loss:  0.015248563
11157 : Training: loss:  0.018629597
11158 : Training: loss:  0.01919156
11159 : Training: loss:  0.01305

11344 : Training: loss:  0.037819955
11345 : Training: loss:  0.023696413
11346 : Training: loss:  0.025036292
11347 : Training: loss:  0.031272475
11348 : Training: loss:  0.041031446
11349 : Training: loss:  0.029253386
11350 : Training: loss:  0.019970246
11351 : Training: loss:  0.012578867
11352 : Training: loss:  0.022851612
11353 : Training: loss:  0.021844901
11354 : Training: loss:  0.019197104
11355 : Training: loss:  0.01598268
11356 : Training: loss:  0.021817006
11357 : Training: loss:  0.008238339
11358 : Training: loss:  0.02254189
11359 : Training: loss:  0.014272268
11360 : Training: loss:  0.041716788
Validation: Loss:  0.052326106  Accuracy:  0.78846157
11361 : Training: loss:  0.009472215
11362 : Training: loss:  0.01209569
11363 : Training: loss:  0.032307178
11364 : Training: loss:  0.027221918
11365 : Training: loss:  0.014320596
11366 : Training: loss:  0.021425495
11367 : Training: loss:  0.045546655
11368 : Training: loss:  0.012199265
11369 : Training: loss: 

11578 : Training: loss:  0.026163314
11579 : Training: loss:  0.02990485
11580 : Training: loss:  0.030704223
Validation: Loss:  0.051375028  Accuracy:  0.75
11581 : Training: loss:  0.019543258
11582 : Training: loss:  0.03777456
11583 : Training: loss:  0.029756201
11584 : Training: loss:  0.011082359
11585 : Training: loss:  0.021794591
11586 : Training: loss:  0.05751558
11587 : Training: loss:  0.023745032
11588 : Training: loss:  0.029306622
11589 : Training: loss:  0.017463224
11590 : Training: loss:  0.03287536
11591 : Training: loss:  0.03208247
11592 : Training: loss:  0.0103540085
11593 : Training: loss:  0.014858192
11594 : Training: loss:  0.016352715
11595 : Training: loss:  0.015011812
11596 : Training: loss:  0.010914676
11597 : Training: loss:  0.02052107
11598 : Training: loss:  0.015550422
11599 : Training: loss:  0.017002478
11600 : Training: loss:  0.050709113
Validation: Loss:  0.051427696  Accuracy:  0.7692308
11601 : Training: loss:  0.018374978
11602 : Training

KeyboardInterrupt: 